# Optimization Algorithms
#### Author: Ratan Madankumar Singh                                                                                                                              
#### 16th February 2018

In this document, I have implemented few most commonly used cost functions used for regression and corresponding parameter updating algorithms. We will check the effect of each of these algorithms on a particular dataset to have a comparison between the optimal value and rate of convergence of these algorithms. Here I have implemented 

- **Batch Gradient Descent**
- **Gradient Descent with Momentum**
- **RMS Propagation**
- **Adam**

These algorithms have their parameter initialization functions as well. We will check the rate of convergence using an example of Single perceptron model applied to a dataset of credit card fraud detection. 

### implementation of Single Perceptron Model

In [154]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

def sigmoid(z):
    y_hat = 1.0/(1+np.exp(-z))
    return y_hat

def weightInit(n):
    W = np.random.random((1,n))
    b = 0
    return [W,b]

def forwardPropagation(W,X,b):
    z = np.dot(W,X)+b
    y_hat = sigmoid(z)
    return y_hat

def computeCost(y_hat,y,lambd,W):
    m = len(np.squeeze(y_hat))
    J = np.sum((y_hat-y)**2) + lambd*np.sum(np.array(W)**2)
    return J

def computeGradient(y_hat,y,X,lambd,W):
    m = len(np.squeeze(y_hat))
    dW = np.dot((y_hat-y),X.T)/m + lambd*np.array(W)/m
    db = np.sum(y_hat-y)/m
    return [dW,db]

def predict(W,b,X):
    z = np.dot(W,X)+b
    y_hat = sigmoid(z)
    y_hat = (y_hat > 0.5)*1.0
    return y_hat

def accuracy(y_hat,y):
    acc = np.sum(y_hat == y)*100.0/len(np.squeeze(y))
    return(acc)

## Implementation of Optimization Algorithms
Now we are done with the writing the nitty gritty of code needed for our single Perceptron model to implement. Now we can implement our optimization algorithms actually. These optimization algorithms require some other hyperparameters as well which we need to initialize seperately.  

In [145]:
# Gradient Descent Algorithm

def gradientDescent(W,b,dW,db,alpha):
    W = W - alpha*dW
    b = b - alpha*db
    return [W,b]

# Gradient Descent with Momentum

def initMomentum(W,b):
    Vw = np.zeros(W.shape)
    Vb = 0
    return [Vw,Vb]

def gradientDescentMomentum(W,b,dW,db,Vw,Vb,beta,alpha):
    Vw = beta*Vw + (1-beta)*dW
    Vb = beta*Vb + (1-beta)*db
    W = W - alpha*Vw
    b = b - alpha*Vb
    return [W,b,Vw,Vb]

# RMS Propagation

def initRMS(W,b):
    Sw = np.zeros(W.shape)
    Sb = 0
    return [Sw,Sb]

def RMSProp(W,b,dW,db,Sw,Sb,beta,alpha,epsilon = 1e-8):
    Sw = beta*Sw + (1-beta)*np.square(dW)
    Sb = beta*Sb + (1-beta)*np.square(db)
    W = W - alpha*dW/(np.sqrt(Sw)+epsilon)
    b = b - alpha*db/(np.sqrt(Sb)+epsilon)
    return [W,b,Sw,Sb]

# Adam

def initAdam(W,b):
    Sw = np.zeros(W.shape)
    Sb = 0
    Vw = np.zeros(W.shape)
    Vb = 0
    return [Sw,Sb,Vw,Vb]

def adam(W,b,dW,db,Sw,Sb,Vw,Vb,beta1,beta2,alpha,epsilon = 1e-8):
    Sw = beta1*Sw + (1-beta1)*np.square(dW)
    Sb = beta1*Sb + (1-beta1)*np.square(db)
    Vw = beta2*Vw + (1-beta2)*dW
    Vb = beta2*Vb + (1-beta2)*db
    W = W - alpha*Vw/(np.sqrt(Sw)+epsilon)
    b = b - alpha*Vb/(np.sqrt(Sb)+epsilon)
    return [W,b,Sw,Sb,Vw,Vb]


## Checking our optimization Algorithm Performance
Now we will load our dataset. This dataset has been taken from kaggle and contains different parameters to detect a credit card fraud detection. The fraudulent transactions has been labeled as '1' and rest as '0'. Since we are just observing performance of our algorithms, we will not discuss our dataset in detail. I have stored dataset in my local machine. Before applying machine learning algorithm, we will split dataset. 

In [146]:
file_path = "C:\\Users\\Ratan Singh\\Documents\\Neural Network\\Optimization Algorithms\\creditcard.csv"
OriginalDataFrame = pd.read_csv(file_path)
print("Dimensions of file is "+str(DataFrame.shape[0])+" rows and "+str(DataFrame.shape[1])+" columns" )

Dimensions of file is 284807 rows and 30 columns


Since we don't need a column "Time" so we will drop it from the dataset. Also we are splitting our data into trainData and testData with 200000 rows in trainData. Before spliting we are randomizing the data so that both trainData and testData comes from the same distribution and our model is not affected by bias. Also here we are setting some more hyperparameters like **number of iterations (num_iter)**, **learning rate (alpha) ** , **Regularization Parameter (lambd)**  and setting the basic flow of learning. To store the results of different optimization algorithm, we initialize different cost functions, different parameters and different gradient.

In [147]:
DataFrame = OriginalDataFrame.drop(["Time"],axis = 1)
order = np.random.permutation(DataFrame.shape[0])
print("Dimensions of file is "+str(DataFrame.shape[0])+" rows and "+str(DataFrame.shape[1])+" columns" )

DataFrame = DataFrame.iloc[order,:]

cutOff = 200000

X_trainData = DataFrame.iloc[range(0,cutOff),range(0,29)].T
X_testData = DataFrame.iloc[range(cutOff,DataFrame.shape[0]),range(0,29)].T

Y_trainData = (DataFrame.iloc[range(0,cutOff),29].T).values.reshape(1,cutOff)
Y_testData = (DataFrame.iloc[range(cutOff,DataFrame.shape[0]),29].T).values.reshape(1,DataFrame.shape[0]-cutOff)

num_iter = 1500
alpha = 0.1
lambd = 0.1
beta = 0.9
beta1 = 0.99
beta2 = 0.9

J_bgd = []
J_gdm = []
J_RMS = []
J_adm = []

[W_init,b_init] = weightInit(29)

[W_bgd,b_bgd] = [W_init,b_init]
[W_gdm,b_gdm] = [W_init,b_init]
[W_RMS,b_RMS] = [W_init,b_init]
[W_adm,b_adm] = [W_init,b_init]

[Vw_gdm,Vb_gdm] = initMomentum(W_gdm,b_gdm)
[Sw_RMS,Sb_RMS] = initRMS(W_RMS,b_RMS)
[Sw_adm,Sb_adm,Vw_adm,Vb_adm] = initAdam(W_adm,b_adm)


for i in range(0,num_iter):
    
    y_hat_bgd = forwardPropagation(W_bgd,X_trainData,b_bgd)
    y_hat_gdm = forwardPropagation(W_gdm,X_trainData,b_gdm)
    y_hat_RMS = forwardPropagation(W_RMS,X_trainData,b_RMS)
    y_hat_adm = forwardPropagation(W_adm,X_trainData,b_adm)
    
    if(i%100 == 0):
        print("ITERATION::"+str(i))
        J_bgd.append(computeCost(y_hat_bgd,Y_trainData,lambd,W_bgd))
        J_gdm.append(computeCost(y_hat_gdm,Y_trainData,lambd,W_gdm))
        J_RMS.append(computeCost(y_hat_RMS,Y_trainData,lambd,W_RMS))
        J_adm.append(computeCost(y_hat_adm,Y_trainData,lambd,W_adm))
    
    [dW_bgd,db_bgd] = computeGradient(y_hat_bgd,Y_trainData,X_trainData,lambd,W_bgd)
    [dW_gdm,db_gdm] = computeGradient(y_hat_gdm,Y_trainData,X_trainData,lambd,W_gdm)
    [dW_RMS,db_RMS] = computeGradient(y_hat_RMS,Y_trainData,X_trainData,lambd,W_RMS)
    [dW_adm,db_adm] = computeGradient(y_hat_adm,Y_trainData,X_trainData,lambd,W_adm)
    
    [W_bgd,b_bgd] = gradientDescent(W_bgd,b_bgd,dW_bgd,b_bgd,alpha)
    [W_gdm,b_gdm,Vw_gdm,Vb_gdm] = gradientDescentMomentum(W_gdm,b_gdm,dW_gdm,db_gdm,Vw_gdm,Vb_gdm,beta,alpha)
    [W_RMS,b_RMS,Sw_RMS,Sb_RMS] = RMSProp(W_RMS,b_RMS,dW_RMS,db_RMS,Sw_RMS,Sb_RMS,beta,alpha)
    [W_adm,b_adm,Sw_adm,Sb_adm,Vw_adm,Vb_adm] = adam(W_adm,b_adm,dW_adm,db_adm,Sw_adm,Sb_adm,Vw_adm,Vb_adm,beta1,beta2,alpha)
        
print("Training the model is completed ::")
    


Dimensions of file is 284807 rows and 30 columns
ITERATION::0
ITERATION::100
ITERATION::200
ITERATION::300
ITERATION::400
ITERATION::500
ITERATION::600
ITERATION::700
ITERATION::800
ITERATION::900
ITERATION::1000
ITERATION::1100
ITERATION::1200
ITERATION::1300
ITERATION::1400
Training the model is completed ::


In [148]:
from prettytable import PrettyTable
table = PrettyTable()
table.add_column('Gradient Descent',J_bgd)
table.add_column('Gradient Descent Momentum',J_gdm)
table.add_column('RMS Propagation',J_RMS)
table.add_column('Adam',J_adm)
print(table)     

+------------------+---------------------------+-----------------+---------------+
| Gradient Descent | Gradient Descent Momentum | RMS Propagation |      Adam     |
+------------------+---------------------------+-----------------+---------------+
|  182854.335881   |       182854.335881       |  182854.335881  | 182854.335881 |
|  1449.83039971   |       1219.86143036       |  199.629982152  | 214.405246009 |
|  1649.49566123   |       1132.61488642       |  3276.16160607  | 217.377575734 |
|  2725.20680498   |       1055.22523983       |  180.663596288  | 220.254575133 |
|  5998.32547266   |       979.504068281       |   175.34524058  | 209.156665238 |
|  5148.27747178   |       904.055347839       |  174.238366628  | 197.767441804 |
|  3948.53477997   |       924.738043288       |  186.619005844  | 192.349844329 |
|  3223.39882396   |       1283.37727125       |  132.116528537  | 164.705225425 |
|  2789.79126546   |       2787.39988929       |  177.380202071  | 165.095513588 |
|  2

In [156]:
y_hat_bgd = predict(W_bgd,b_bgd,X_testData)
y_hat_gdm = predict(W_gdm,b_gdm,X_testData)
y_hat_RMS = predict(W_RMS,b_RMS,X_testData)
y_hat_adm = predict(W_adm,b_adm,X_testData)

print("Accuracy due to gradient Descent :: "+str(accuracy(y_hat_bgd,Y_testData)))
print("Accuracy due to gradient Descent with Momentum :: "+str(accuracy(y_hat_gdm,Y_testData)))
print("Accuracy due to RMS Propagation :: "+str(accuracy(y_hat_RMS,Y_testData)))
print("Accuracy due to Adam :: "+str(accuracy(y_hat_adm,Y_testData)))

Accuracy due to gradient Descent :: 99.6120603252
Accuracy due to gradient Descent with Momentum :: 99.8891600929
Accuracy due to RMS Propagation :: 99.9127430519
Accuracy due to Adam :: 99.9068473121
